# Text Summarising of the articles

## Text-Summarization
**Automatic summarization** is the process of shortening a text document with software, in order to create a summary with the major points of the original document. Technologies that can make a coherent summary take into account variables such as length, writing style and syntax.

Automatic data summarization is part of the real machine learning and data mining. The main idea of summarization is to find a subset of data which contains the "information" of the entire set. Such techniques are widely used in industry today. Search engines are an example; others include summarization of documents, image collections and videos. Document summarization tries to create a representative summary or abstract of the entire document, by finding the most informative sentences, while in image summarization the system finds the most representative and important (i.e. salient) images. For surveillance videos, one might want to extract the important events from the uneventful context.

There are two general approaches to automatic summarization: Extraction and Abstraction. 
1. *Extractive Summarization*: These methods rely on extracting several parts, such as phrases and sentences, from a piece of text and stack them together to create a summary. Therefore, identifying the right sentences for summarization is of utmost importance in an extractive method.
2. *Abstractive Summarization*: These methods use advanced NLP techniques to generate an entirely new summary. Some parts of this summary may not even appear in the original text. Such a summary might include verbal innovations. 
Research to date has focused primarily on extractive methods, which are appropriate for image collection summarization and video summarization.

In this Jupyter notebook, TextRank algorithm for extractive text summarization is implemented using Google's PageRank search algorithm to generate corelations among sentences.

### Libraries Used
- [Numpy](http://www.numpy.org)
- [Pandas](https://pandas.pydata.org/)
- [Natural Language Toolkit](https://www.nltk.org/)

### Algorithms and Concepts
- TextRank
- [PageRank](https://en.wikipedia.org/wiki/PageRank)
- [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

GloVe word embeddings are vector representation of words. These word embeddings will be used to create vectors for our sentences.
We will be using the pre-trained Wikipedia 2014 + Gigaword 5 GloVe vectors available [here](http://www.kaggle.com/mohitkundu1/glove-word-embeddings). Heads up – the size of these word embeddings is 331 MB.

In [ ]:
word_embeddings = {}
file = open('../input/glove-word-embeddings/glove.6B.100d.txt', encoding='utf-8')
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
file.close()
len(word_embeddings)

As the above file is big it will take some time to create the word embeddings.

In [ ]:
df = pd.read_excel('../input/medicine-descrptions/TASK.xlsx')
df

Let us now see the names of the columns.

In [ ]:
df.columns

Let's change the name of the column and we can delete the fisrt row as it will of no use to us.

In [ ]:
df.rename(columns = {'Unnamed: 1' : 'Introduction' }, inplace=True)
df.drop(0)

Now this looks good. Let us now convert the dataset into a dictionary as it would be easy to use.

In [ ]:
text_dictionary = {}
for i in range(1,len(df['TEST DATASET'])):
    text_dictionary[i] = df['Introduction'][i]   
print(text_dictionary[1])

**There are 1000 such description of the different medicines. The task is to give summarised form of these description.**

In [ ]:
def remove_stopwords(sen):
    stop_words = stopwords.words('english')
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
def sentence_vector_func (sentences_cleaned) : 
    sentence_vector = []
    for i in sentences_cleaned:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vector.append(v)
    return (sentence_vector)

In [ ]:
def summary_text (test_text, n = 5):
    sentences = []
    sentences.append(sent_tokenize(test_text))
    sentences = [y for x in sentences for y in x] 
    clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    sentence_vectors = sentence_vector_func(clean_sentences)
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)))
    summarised_string = ''
    for i in range(n):
        try:
            summarised_string = summarised_string + str(ranked_sentences[i][1])            
        except IndexError:
            print ("Summary Not Available")
    return (summarised_string)

In [ ]:
print("Kindly let me know in how many sentences you want the summary - ")
x = 3
summary_dictionary = {}
for key in text_dictionary:
    para = text_dictionary[key]    
    summary = summary_text(para,x)
    summary_dictionary[key] = summary
    if key>0 and key<=10 :
        print("Summary of the article - ",key)
        print(summary)
        print('='*50)    
print ("*"*10,"The process has been completed successfully","*"*10)

In [ ]:
summary_table = pd.DataFrame(list(summary_dictionary.items()),columns = ['TEST DATASET','Summary'])
data_table = pd.DataFrame(list(text_dictionary.items()),columns = ['TEST DATASET','Introduction'])
# Combining the findings into the table
result  = pd.concat([data_table , summary_table['Summary']], axis = 1 , sort = False)
result